Để hiểu cách mà `recommendQ.tsx` và `ChatbotWidget.tsx` liên kết với nhau, chúng ta sẽ xem xét từng phần của mã và cách chúng tương tác thông qua các sự kiện và hàm.

### 1. Cấu trúc của `recommendQ.tsx`

Trong file `recommendQ.tsx`, chúng ta có một component React có tên là `RecommendQ`. Component này hiển thị một danh sách các câu hỏi mà người dùng có thể click vào. Khi người dùng click vào một câu hỏi, một sự kiện sẽ được dispatch để gửi câu hỏi và context đến chatbot.

#### Mã của `recommendQ.tsx`

```typescript
import React from 'react';
import Button from '@/app/components/base/button';
import { openChatbot } from '@/app/components/mindpal/chatbot/ChatbotWidget';

type Props = {
  nodeExecution: Record<string, any>
  isShowTitle?: boolean
}

const SHARED_CONTEXT = "AI's role and impact in modern education";

const DEFAULT_QUESTIONS = [
  { question: "How can AI improve personalized learning experiences?" },
  { question: "What are the ethical considerations of using AI in education?" },
  { question: "Can AI help in tracking student progress more effectively?" }
];

export const RecommendQ: React.FC<Props> = ({ nodeExecution, isShowTitle = false }) => {
  const handleQuestionClick = (question: string) => {
    openChatbot(); // Mở chatbot khi click vào câu hỏi
    
    console.log('Dispatching question:', question); // Log câu hỏi
    console.log('Dispatching context:', SHARED_CONTEXT); // Log context

    // Dispatch event sendQuestion với câu hỏi và context
    window.dispatchEvent(new CustomEvent('sendQuestion', {
      detail: {
        question,
        context: SHARED_CONTEXT
      }
    }));
  };

  return (
    <>
      {isShowTitle && (
        <div className="flex justify-center items-center flex-col gap-2 mt-4">
          <span className="font-semibold">Recommended Questions</span>
        </div>
      )}
      <div className="flex flex-col gap-2 mt-2">
        {DEFAULT_QUESTIONS.map((q, index) => (
          <Button
            key={index}
            className="text-left px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 rounded-lg"
            onClick={() => handleQuestionClick(q.question)} // Gọi hàm khi click
          >
            {q.question}
          </Button>
        ))}
      </div>
    </>
  );
};

export default RecommendQ;
```

### 2. Cấu trúc của `ChatbotWidget.tsx`

Trong file `ChatbotWidget.tsx`, chúng ta có một component React có tên là `ChatbotWidget`. Component này chịu trách nhiệm khởi tạo chatbot và lắng nghe các sự kiện từ `recommendQ.tsx`. Khi nhận được sự kiện `sendQuestion`, nó sẽ xử lý câu hỏi và context được gửi từ `recommendQ.tsx`.

#### Mã của `ChatbotWidget.tsx`

```typescript
import React, { useEffect, useRef, useState } from 'react';

export const openChatbot = () => {
  window.dispatchEvent(new CustomEvent('openChatbot')); // Mở chatbot
};

export const ChatbotWidget: React.FC = () => {
  const [error, setError] = useState<string | null>(null);
  const scriptRef = useRef<HTMLScriptElement | null>(null);
  const initAttempted = useRef(false);

  useEffect(() => {
    if (typeof window === 'undefined' || initAttempted.current) return;

    const handleQuestion = (event: Event) => {
      const questionEvent = event as SendQuestionEvent;
      const { question, context } = questionEvent.detail;

      console.log('Received question:', question); // Log câu hỏi nhận được
      console.log('Received context:', context); // Log context nhận được

      if (window.chatbot?.sendMessage) {
        console.log('Chatbot is ready, sending message...'); // Log khi chatbot sẵn sàng
        const message = `${context}\n\n${question}`; // Gộp context và câu hỏi
        window.chatbot.sendMessage(message); // Gửi tin nhắn đến chatbot
        console.log('Message sent to chatbot:', message); // Log tin nhắn đã gửi
      } else {
        console.error('Chatbot not ready'); // Log lỗi nếu chatbot chưa sẵn sàng
      }
    };

    const handleOpenChatbot = () => {
      window.initChatbot?.({
        position: 'right',
        width: '400px',
        height: '600px'
      });
    };

    window.addEventListener('openChatbot', handleOpenChatbot); // Lắng nghe sự kiện mở chatbot
    window.addEventListener('sendQuestion', handleQuestion); // Lắng nghe sự kiện gửi câu hỏi

    const initChatbot = async () => {
      // Khởi tạo chatbot
    };

    initChatbot();

    return () => {
      window.removeEventListener('openChatbot', handleOpenChatbot);
      window.removeEventListener('sendQuestion', handleQuestion);
      if (scriptRef.current?.parentNode) {
        scriptRef.current.parentNode.removeChild(scriptRef.current);
      }
    };
  }, []);

  if (error) {
    return (
      <div className="rounded bg-red-50 p-4 text-sm text-red-600">
        Failed to load chatbot: {error}
      </div>
    );
  }

  return <div id="chatbot-container" className="h-full w-full" />;
};

export default ChatbotWidget;
```

### 3. Cách Liên Kết Giữa Hai Component

- **Khi người dùng click vào một câu hỏi trong `RecommendQ`**:
  - Hàm `handleQuestionClick` được gọi.
  - Chatbot được mở thông qua `openChatbot()`.
  - Một sự kiện `sendQuestion` được dispatch với chi tiết là câu hỏi và context.

- **Trong `ChatbotWidget`**:
  - Sự kiện `sendQuestion` được lắng nghe.
  - Khi sự kiện này được nhận, hàm `handleQuestion` sẽ được gọi.
  - Câu hỏi và context được lấy từ sự kiện và gộp lại thành một tin nhắn.
  - Tin nhắn này sau đó được gửi đến chatbot thông qua `window.chatbot.sendMessage`.

### Kết luận

Hai component này liên kết với nhau thông qua việc sử dụng các sự kiện của `window`. `RecommendQ` dispatch một sự kiện khi người dùng click vào câu hỏi, và `ChatbotWidget` lắng nghe sự kiện đó để xử lý và gửi câu hỏi đến chatbot. Điều này cho phép bạn tách biệt logic giữa việc hiển thị câu hỏi và xử lý câu hỏi trong chatbot, giúp mã dễ bảo trì và mở rộng hơn.

File `types.ts` trong dự án của bạn định nghĩa các kiểu dữ liệu (type definitions) cho các đối tượng và sự kiện liên quan đến chatbot. Đây là một phần quan trọng trong việc sử dụng TypeScript, giúp đảm bảo rằng các đối tượng và hàm được sử dụng đúng cách trong mã nguồn. Dưới đây là phân tích chi tiết về nội dung của file `types.ts` mà bạn đã cung cấp:

### 1. Định nghĩa `ChatbotConfig`

```typescript
interface ChatbotConfig {
    position: 'right' | 'left'; // Vị trí của chatbot (có thể là bên phải hoặc bên trái)
    width: string; // Chiều rộng của chatbot
    height: string; // Chiều cao của chatbot
}
```

- **Mục đích**: Định nghĩa cấu hình cho chatbot, bao gồm vị trí, chiều rộng và chiều cao. Điều này giúp bạn dễ dàng cấu hình chatbot khi khởi tạo.

### 2. Định nghĩa `Window` Interface

```typescript
declare global {
    interface Window {
        initChatbot?: (config: ChatbotConfig) => void; // Hàm khởi tạo chatbot
        chatbot?: {
            sendMessage: (message: string, context?: string) => void; // Hàm gửi tin nhắn đến chatbot
        }
    }
}
```

- **Mục đích**: Mở rộng interface `Window` để thêm các thuộc tính và phương thức liên quan đến chatbot. Điều này cho phép bạn gọi `window.initChatbot` và `window.chatbot.sendMessage` mà TypeScript sẽ nhận diện và kiểm tra kiểu dữ liệu.

### 3. Định nghĩa `ChatMessageEvent`

```typescript
interface ChatMessageEvent extends CustomEvent {
    detail: {
        message: string; // Tin nhắn được gửi đến chatbot
        context?: string; // Ngữ cảnh (có thể không có)
    }
}
```

- **Mục đích**: Định nghĩa kiểu cho sự kiện `ChatMessageEvent`, cho phép bạn gửi tin nhắn và ngữ cảnh đến chatbot thông qua một sự kiện tùy chỉnh. Điều này giúp bạn dễ dàng xử lý các sự kiện liên quan đến tin nhắn trong mã nguồn.

### 4. Định nghĩa `SendQuestionEvent`

```typescript
interface SendQuestionEvent extends CustomEvent {
    detail: {
        question: string; // Câu hỏi được gửi đến chatbot
        context: string; // Ngữ cảnh liên quan đến câu hỏi
    }
}
```

- **Mục đích**: Định nghĩa kiểu cho sự kiện `SendQuestionEvent`, cho phép bạn gửi câu hỏi và ngữ cảnh đến chatbot thông qua một sự kiện tùy chỉnh. Điều này giúp bạn dễ dàng xử lý các sự kiện liên quan đến câu hỏi trong mã nguồn.

### Kết luận

File `types.ts` cung cấp các định nghĩa kiểu dữ liệu cần thiết cho việc tương tác với chatbot trong ứng dụng của bạn. Việc sử dụng TypeScript giúp bạn phát hiện lỗi sớm hơn trong quá trình phát triển, đảm bảo rằng các đối tượng và hàm được sử dụng đúng cách, và cải thiện khả năng bảo trì mã nguồn.